# 性能建议

+ 编写类型稳定的代码
  - Julia程序执行分为编译期和执行期，编译期就能确定变量类型的代码为类型稳定的代码， 如果必须要程序真正跑起来才能确定变量类型，那说明类型不稳定，可以用`@code_warntype`检查
  - Julia的编译器会自动进行类型推断，一般来说不需要进行类型标注，除了含糊不清的情况
+ 避免内存分配
  - 必要时使用数组的视图，而不是copy
+ 并行计算(适用于计算顺序不影响最终结果的for-loop中)
  - `@simd` 单指令多数据
  - `@turbo` 进阶版的`@simd`， 可以替代`@inbounds @simd`
  - `@threads` 单进程多线程，共享一块内存区域
  - 多进程与分布式数据（或者copy数据到所有其它julia进程），每个julia程序单独享用一块内存区域
  - 以上几种， 优先使用`@turbo`， 其次使用多线程

## 避免定义全局变量
+ 避免使用`global x = 0`来定义全局变量
+ 使用`const x = 0`定义全局常量对性能是有益的

## @btime进行性能比较

`@btime`用来判断程序执行时间和内存分配，julia自带的`@time`第一次执行加入了编译时间，建议使用`@btime`多次执行`@benchmark`后取的最小时间，不包括编译时间，可能会比较慢

In [1]:
using BenchmarkTools

In [2]:
@btime mapreduce(x->sin(x), +, 1:10)

  49.443 ns (0 allocations: 0 bytes)


1.4111883712180104

In [3]:
@benchmark mapreduce(x->sin(x), +, 1:10)

BenchmarkTools.Trial: 10000 samples with 987 evaluations.
 Range (min … max):  51.773 ns … 125.228 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     51.976 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.496 ns ±   3.199 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▃▄▂▂ ▂                                                     ▁
  █████████▇█▆▆▅▅▅▅▅▄▅▃▃▄▁▅▄▆▆█▇▆▇▆▆▅▇▅▆▆▅▆▃▅▃▅▄▄▄▄▅▃▄▃▁▅▁▁▁▁▄ █
  51.8 ns       Histogram: log(frequency) by time      61.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

## @code_warntype判断类型是否稳定

In [4]:
function mysum(A)
    s = 0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [5]:
a = rand(10^7)
@code_warntype mysum(a)

MethodInstance for mysum(::Vector{Float64})
  from mysum(A) in Main at In[4]:1
Arguments
  #self#::Core.Const(mysum)
  A::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Union{Float64, Int64}
  a::Float64
Body::Union{Float64, Int64}
1 ─       (s = 0)
│   %2  = A::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (a = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (s = s + a)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s



关注输出中的红色字体，红色表示类型不稳定的代码声明, `s`既可能是Float64也可能是Int64类型，类型不稳定

In [6]:
function mysum2(A)
    s = zero(A)
    for a in A
        s += a
    end
    return s
end

mysum2 (generic function with 1 method)

In [7]:
@code_warntype mysum2(rand(10))

MethodInstance for mysum2(::Vector{Float64})
  from mysum2(A) in Main at In[6]:1
Arguments
  #self#::Core.Const(mysum2)
  A::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Vector{Float64}
  a::Float64
Body::Vector{Float64}
1 ─      (s = Main.zero(A))
│   %2 = A::Vector{Float64}
│        (@_3 = Base.iterate(%2))
│   %4 = (@_3 === nothing)::Bool
│   %5 = Base.not_int(%4)::Bool
└──      goto #3 if not %5
2 ─ %7 = @_3::Tuple{Float64, Int64}
│        (a = Core.getfield(%7, 1))
│        Core.getfield(%7, 2)
│        (s = s + a)
│        Core.Const(:(@_3 = Base.iterate(%2, %9)))
│        Core.Const(:(@_3 === nothing))
│        Core.Const(:(Base.not_int(%12)))
│        Core.Const(:(goto %16 if not %13))
└──      Core.Const(:(goto %7))
3 ┄      return s



## 广播VS循环

R和Matlab的向量化代码比较快的原因是
+ 向量化代码用C语言展开为for-loop，不受到R或Matlab本身的限制
+ 底层使用C时，数值类型明确，可以触发更加高效的优化手段（如simd）

In [8]:
N = 1000
a = randn(N)
b = randn(N)
c = rand(N)
d = randn(N) * 2;

In [9]:
function testdot(a, b, c, d)
    return sum(a .* b .+ c./ d .- 1) 
end

testdot (generic function with 1 method)

In [10]:
@btime testdot($a, $b, $c, $d)

  1.170 μs (1 allocation: 7.94 KiB)


-1005.6076372449912

In [11]:
function testloop(a, b, c, d)
    s = zero(eltype(a))
    @inbounds @simd for i in eachindex(a)
        s += a[i] * b[i] + c[i] / d[i] - 1
    end
    return s;
end

@btime testloop($a, $b, $c, $d)

  488.601 ns (0 allocations: 0 bytes)


-1005.6076372449912

注意这里的for-loop版本的计算很明显要快于向量化计算，原因在于这么几个方面
+ 向量化代码需要更高的内存占用，储存的中间变量为矩阵
+ for-loop使用`@inbounds`，避免了数组边界检查（配合`eachindex`可以确保不会越界）
+ for-loop使用`@simd`，CPU级别的并行，意思是`单指令多数据`，在有AVX指令的Intel CPU的效果最好，原理在于，寄存器较大，一次运算可以同时处理载入的4批数据（4 * 64= 256 bit），注意事项：用在单层循环，循环的顺序对结果没有影响时才可以使用
+ **不使用`@simd` `@inbounds`的循环性能不如向量化版本**

在实际使用中， 向量化操作简洁易懂， 如果把向量化操作写成for-loop的形式，可以单独把它写成一个函数，这样能让程序更简洁

In [61]:
@code_llvm testloop(a, b, c, d)

;  @ In[11]:1 within `testloop`
; Function Attrs: uwtable
define double @julia_testloop_7763({}* nonnull align 16 dereferenceable(40) %0, {}* nonnull align 16 dereferenceable(40) %1, {}* nonnull align 16 dereferenceable(40) %2, {}* nonnull align 16 dereferenceable(40) %3) #0 {
top:
;  @ In[11]:3 within `testloop`
; ┌ @ simdloop.jl:69 within `macro expansion`
; │┌ @ abstractarray.jl:324 within `eachindex` @ abstractarray.jl:334
; ││┌ @ array.jl:215 within `length`
     %4 = bitcast {}* %0 to { i8*, i64, i16, i16, i32 }*
     %5 = getelementptr inbounds { i8*, i64, i16, i16, i32 }, { i8*, i64, i16, i16, i32 }* %4, i64 0, i32 1
     %6 = load i64, i64* %5, align 8
; └└└
; ┌ @ simdloop.jl:72 within `macro expansion`
; │┌ @ int.jl:83 within `<`
    %.not = icmp eq i64 %6, 0
; │└
   br i1 %.not, label %L37, label %L13.lr.ph

L13.lr.ph:                                        ; preds = %top
   %7 = bitcast {}* %0 to double**
   %8 = load double*, double** %7, align 8
   %9 = bitcast {}* %1 to 

``` julia
@llvm.experimental.vector.reduce.v2.fadd.f64.v4f64(double 0.000000e+00, <4 x double> %bin.rdx)
```
`@code_llvm`的结果里要是有上边那一行基本上就可以确定`@simd`发挥了作用

`@simd`也有一些问题， 据我发现只能用于**加减乘除**操作， 当计算中有`sin, exp`这种形式时无法起作用

LoopVectorization.jl包提供了`@turbo`宏， 这个宏命令提供的加速更加强大， 原理好像和`@simd`差不多

## `@turbo`

In [13]:
using LoopVectorization

> “This library provides the `@turbo` macro, which may be used to **prefix a for loop or broadcast statement**. It then tries to vectorize the loop to improve runtime performance.”

### 加速循环

In [14]:
function noturbo(x1, x2, x3, x4)
    s = zero(eltype(x1))
    for i in eachindex(x1)
        s+= exp(x1[i]) * sin(x2[i]) + x3[i] / x4[i] + 1
    end
    return s
end
function withturbo(x1, x2,x3, x4)
    s = zero(eltype(x1))
    @turbo for i in eachindex(x1)
        s+= exp(x1[i]) * sin(x2[i]) + x3[i] / x4[i] + 1
    end
    return s
end

withturbo (generic function with 1 method)

In [15]:
N = 10000
x1 = randn(N)
x2 = randn(N)
x3 = randn(N)
x4 = randn(N);

In [16]:
@btime noturbo($x1, $x2, $x3, $x4)

  144.700 μs (0 allocations: 0 bytes)


11704.6471947779

In [17]:
@btime withturbo($x1, $x2, $x3, $x4)

  25.900 μs (0 allocations: 0 bytes)


11704.647194777895

他可以加速`sin, exp`， 确实比`@simd`强大了很多

### 加速广播

In [18]:
f_noturbo(x1, x2, x3, x4) = sum(x1 .* x2 .* x3 ./ exp.(x4))
f_turbo(x1, x2, x3, x4) = sum(@turbo x1 .* x2 .* x3 ./ exp.(x4))
f2_turbo(x1, x2, x3, x4) = @turbo sum(x1 .* x2 .* x3 ./ exp.(x4))

f2_turbo (generic function with 1 method)

In [19]:
@btime f_noturbo($x1, $x2, $x3, $x4)

  63.900 μs (2 allocations: 78.17 KiB)


-295.24468897233237

In [20]:
@btime f_turbo($x1, $x2, $x3, $x4)

  13.100 μs (2 allocations: 78.17 KiB)


-295.24468897233214

In [21]:
@btime f2_turbo($x1, $x2, $x3, $x4)

  13.300 μs (2 allocations: 78.17 KiB)


-295.24468897233214

确实是很牛， 广播也能加速的很快

### 额外测试

在实际应用中， 我经常需要计算概率密度值， 看一下这种情况能不能算得快，有加速

<font style="color:red;font-size:18pt"> TL,DR: <br> 连续分布可以有加速， 效果还不错， <br> 离散分布会报错</font>

In [22]:
using Distributions, Random
seed = Random.seed!(2022 - 8 - 9)

TaskLocalRNG()

In [23]:
fd_noturbo(x) = sum(logpdf.(Normal(), x))
fd_turbo(x) = sum(@turbo logpdf.(Normal(), x))

fd_turbo (generic function with 1 method)

In [24]:
X = randn(seed, N);

In [25]:
@btime fd_noturbo($X)

  73.200 μs (2 allocations: 78.17 KiB)


-14260.761394916859

In [26]:
@btime fd_turbo($X)

  25.300 μs (2 allocations: 78.17 KiB)


-14260.761394916859

牛皮， 竟然快了接近三倍， 再试试负二项分布

In [27]:
fd2_noturbo(x) = sum(logpdf.(Poisson(10), x))
fd2_turbo(x) = sum(@turbo logpdf.(Poisson(10), x))

fd2_turbo (generic function with 1 method)

In [28]:
X2 = rand(seed, Poisson(10), N);

In [29]:
@btime fd2_noturbo($X2)

  355.200 μs (2 allocations: 78.17 KiB)


-25611.709792507012

In [30]:
X2 = Vector{Float64}(X2);

In [31]:
using Test

In [63]:
@test_broken fd2_turbo(X2)

Test Broken
  Expression: fd2_turbo(X2)

## Val（）

Val可以给Julia编译器提供额外的类型信息

In [33]:
using StaticArrays

In [34]:
SVector{4, Float64}(1:4)

4-element SVector{4, Float64} with indices SOneTo(4):
 1.0
 2.0
 3.0
 4.0

In [35]:
function static(v::Vector)
    return SVector{length(v), eltype(v)}(v)
end

static (generic function with 1 method)

In [36]:
@code_warntype static([1, 2, 3.0])

MethodInstance for static(::Vector{Float64})
  from static(v::Vector) in Main at In[35]:1
Arguments
  #self#::Core.Const(static)
  v::Vector{Float64}
Body::Any
1 ─ %1 = Main.length(v)::Int64
│   %2 = Main.eltype(v)::Core.Const(Float64)
│   %3 = Core.apply_type(Main.SVector, %1, %2)::Type{SVector{_A, Float64}} where _A
│   %4 = (%3)(v)::Any
└──      return %4



为啥显示类型不稳定？程序在编译时仍然不知道length(v)是多少，运行时才知道，但是`eltype(v)`在编译时是知道的，julia能推断出`eltype(v)`

Julia对于`Val`结构体的定义, 可以看出`Val(X)`实例化了一个结构体， 得到`Val{X}()`
```julia
struct Val{X} where X end
Val(X) = Val{X}()
```

In [37]:
function static2(v::Vector, ::Val{l}) where {l}
    return SVector{l, eltype(v)}(v)
end

static2 (generic function with 1 method)

In [38]:
function static3(v::Vector, l::Int64)
    return SVector{l, eltype(v)}(v)
end

static3 (generic function with 1 method)

In [39]:
@code_warntype static2([1, 2, 3], Val(3))

MethodInstance for static2(::Vector{Int64}, ::Val{3})
  from static2(v::Vector, ::Val{l}) where l in Main at In[37]:1
Static Parameters
  l = 3
Arguments
  #self#::Core.Const(static2)
  v::Vector{Int64}
  _::Core.Const(Val{3}())
Body::SVector{3, Int64}
1 ─ %1 = $(Expr(:static_parameter, 1))::Core.Const(3)
│   %2 = Main.eltype(v)::Core.Const(Int64)
│   %3 = Core.apply_type(Main.SVector, %1, %2)::Core.Const(SVector{3, Int64})
│   %4 = (%3)(v)::SVector{3, Int64}
└──      return %4



上边的代码为啥类型稳定了？ “它将值l作为编译器就可以知道的类型信息告诉julia了” -JonnyChen

In [40]:
@code_warntype static3([1, 2, 3.0], 3)

MethodInstance for static3(::Vector{Float64}, ::Int64)
  from static3(v::Vector, l::Int64) in Main at In[38]:1
Arguments
  #self#::Core.Const(static3)
  v::Vector{Float64}
  l::Int64
Body::Any
1 ─ %1 = Main.eltype(v)::Core.Const(Float64)
│   %2 = Core.apply_type(Main.SVector, l, %1)::Type{SVector{_A, Float64}} where _A
│   %3 = (%2)(v)::Any
└──      return %3



这个为啥类型不稳定呢？ 貌似3是作为一个变量传进去的， 在llvm源码中看到 `_A = 3`这个代码， 应该是要先执行这个代码， 再去生成`SVector（）`

## 避免声明容器内元素为抽象类型

容器包含：数组，矩阵，向量，元组，命名元组，结构体，自定义结构体等

In [41]:
struct badstructs
    apple::AbstractString 
end

In [42]:
struct goodstructs{T<:AbstractString}
    apple::T 
end

In [43]:
typeof(badstructs("apple"))

badstructs

In [44]:
typeof(goodstructs("apple"))

goodstructs{String}

## 避免声明容器内的元素为抽象类型

In [45]:
struct badstruct2
    a::AbstractMatrix
end

In [46]:
struct goodstruct2{T <: AbstractMatrix}
    a::T
end

## 必要时使用数组视图而不是切片

In [47]:
a = rand(1000, 1000);

In [48]:
@btime a[1:10, 3:10];

  122.009 ns (1 allocation: 736 bytes)


In [49]:
@btime @view a[1:10, 3:10];

  30.391 ns (1 allocation: 64 bytes)


In [50]:
@btime a[1:100:1000, 3:30:900];

  434.673 ns (1 allocation: 2.50 KiB)


In [51]:
@btime @view a[1:100:1000, 3:30:900];

  32.362 ns (1 allocation: 80 bytes)


尤其是不连续切片，这个的提升非常明显， 几乎是十几倍的提升

## 不要在函数内部使用全局变量

### 函数内部使用全局变量(Bad)

In [52]:
# Bad way
I = 100
x = randn(I)
function sumI1()
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI1 (generic function with 1 method)

In [53]:
@code_warntype sumI1()

MethodInstance for sumI1()
  from sumI1() in Main at In[52]:4
Arguments
  #self#::Core.Const(sumI1)
Locals
  @_2::Any
  s::Any
  i::Any
Body::Any
1 ─ %1  = Main.eltype(Main.x)::Any
│         (s = Main.zero(%1))
│   %3  = (1:Main.I)::Any
│         (@_2 = Base.iterate(%3))
│   %5  = (@_2 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_2::Any
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Any
│   %11 = s::Any
│   %12 = Base.getindex(Main.x, i)::Any
│         (s = %11 + %12)
│         (@_2 = Base.iterate(%3, %10))
│   %15 = (@_2 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s



`I`和`x`是全局变量， 函数内部使用了全局变量， 运行时程序不会报错， 但是出现了类型不稳定的情况，原因是`s`和`I`在编译时不知道类型， 只有在执行时才知道

### 函数内部使用全局常量(Good)

In [54]:
# Good way 1
const CI = 100
const Cx = randn(CI)
function sumI2()
    s = zero(eltype(Cx))
    for i in 1:CI
        s+=Cx[i]
    end
    return s
end

sumI2 (generic function with 1 method)

In [55]:
@code_warntype sumI2()

MethodInstance for sumI2()
  from sumI2() in Main at In[54]:4
Arguments
  #self#::Core.Const(sumI2)
Locals
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  s::Float64
  i::Int64
Body::Float64
1 ─ %1  = Main.eltype(Main.Cx)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = (1:Main.CI)::Core.Const(1:100)
│         (@_2 = Base.iterate(%3))
│   %5  = (@_2::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %6  = Base.not_int(%5)::Core.Const(true)
└──       goto #4 if not %6
2 ┄ %8  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Float64
│   %12 = Base.getindex(Main.Cx, i)::Float64
│         (s = %11 + %12)
│         (@_2 = Base.iterate(%3, %10))
│   %15 = (@_2 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s



将全局变量变为全局常量， 能够得到编译器充分优化的代码， julia能够在编译期推断出变量的类型

### 函数通过结构体传入数据（Good如果结构体内的字段有类型标注）

In [56]:
# Good way 2
Base.@kwdef mutable struct data
    I::Int = 100                   # 必须进行类型标注
    x::Vector{Float64} = randn(I)  # 必须进行类型标注
end
function sumI3(d::data)
    x, I = d.x, d.I
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI3 (generic function with 1 method)

In [57]:
@code_warntype sumI3(data())

MethodInstance for sumI3(::data)
  from sumI3(d::data) in Main at In[56]:6
Arguments
  #self#::Core.Const(sumI3)
  d::data
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  s::Float64
  I::Int64
  x::Vector{Float64}
  i::Int64
Body::Float64
1 ─ %1  = Base.getproperty(d, :x)::Vector{Float64}
│   %2  = Base.getproperty(d, :I)::Int64
│         (x = %1)
│         (I = %2)
│   %5  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%5))
│   %7  = (1:I)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%7))
│   %9  = (@_3 === nothing)::Bool
│   %10 = Base.not_int(%9)::Bool
└──       goto #4 if not %10
2 ┄ %12 = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%12, 1))
│   %14 = Core.getfield(%12, 2)::Int64
│   %15 = s::Float64
│   %16 = Base.getindex(x, i)::Float64
│         (s = %15 + %16)
│         (@_3 = Base.iterate(%7, %14))
│   %19 = (@_3 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #4 if not %20
3 ─ 

### 函数通过结构体传入数据（Bad如果结构体内的字段没有类型标注）

In [58]:
# Bad way 
Base.@kwdef mutable struct data2
    I = 100            
    x = randn(I) 
end
function sumI4(d::data2)
    x, I = d.x, d.I
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI4 (generic function with 1 method)

In [59]:
@code_warntype sumI4(data2())

MethodInstance for sumI4(::data2)
  from sumI4(d::data2) in Main at In[58]:6
Arguments
  #self#::Core.Const(sumI4)
  d::data2
Locals
  @_3::Any
  s::Any
  I::Any
  x::Any
  i::Any
Body::Any
1 ─ %1  = Base.getproperty(d, :x)::Any
│   %2  = Base.getproperty(d, :I)::Any
│         (x = %1)
│         (I = %2)
│   %5  = Main.eltype(x)::Any
│         (s = Main.zero(%5))
│   %7  = (1:I)::Any
│         (@_3 = Base.iterate(%7))
│   %9  = (@_3 === nothing)::Bool
│   %10 = Base.not_int(%9)::Bool
└──       goto #4 if not %10
2 ┄ %12 = @_3::Any
│         (i = Core.getfield(%12, 1))
│   %14 = Core.getfield(%12, 2)::Any
│   %15 = s::Any
│   %16 = Base.getindex(x, i)::Any
│         (s = %15 + %16)
│         (@_3 = Base.iterate(%7, %14))
│   %19 = (@_3 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #4 if not %20
3 ─       goto #2
4 ┄       return s

